In [4]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [2]:
doc = """
드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
"향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
"""

In [5]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
print(tokenized_nouns)

드론 활용 범위 점차 확대 최근 미세먼지 관리 드론 활용 서울시 미세먼지 계절 관리제 기간 지난달 개 월간 드론 측정 장치 달 미세먼지 집중 관리 실시 드론 산업 단지 사업 밀집 지역 미세먼지 배출 수치 점검 현장 모습 영상 영상 통해 미세먼지 방지 시설 제대로 가동 업체 무허가 시설 대한 단속 한층 전망 드론 활용 가장 적극 소방청 복합 재난 대응 차원 드론 관련 전문 인력 보강 어가 지난해 말 기준 소방청 보유 드론 총 드론 조종 자격증 소방대 경우 명 이 중 실기 평가 지도 자격증 드론 전문가 명도 배치 소방청 관계자 소방 드론 재난 현장 영상 정보 수집 산악 수난 사고 시 인명 수색 구조 활동 유독가스 폭발사고 시 대원 안전 확보 등 활용 며 향후 화재 진압 인명구조 등 드론 활용 위해 연구개발 고 말


In [7]:
n_gram_range = (2, 3)

# n_gram을 추출
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print(len(candidates))
print(candidates[:10])

222
['가동 업체' '가동 업체 무허가' '가장 적극' '가장 적극 소방청' '경우 실기' '경우 실기 평가' '계절 관리제'
 '계절 관리제 기간' '관계자 소방' '관계자 소방 드론']


In [9]:
# 문서와 문서로부터 추출한 키워드를 SBERT를 통해 수치화, 한국어를 포함하고 있는 다국어 SBERT 로드 
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [12]:
# 문서와 가장 유사한 키워드들을 추출 -> 문서를 대표하기 좋은 키워드
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['드론 산업', '드론 드론 조종', '실시 드론 산업', '관리 드론 활용', '미세먼지 관리 드론']


In [11]:
# 다양한 의미의 키워드를 추출하고 싶다면 Max Sum Similarity, Maximal Marginal Relevance를 사용해 키워드를 선정

"""
Max Sum Similarity
데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됨
후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것.
"""

def max_sum_similarity(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    
    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)
    
    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]
    
    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim
    
    return [words_vals[idx] for idx in candidate]

In [14]:
# 상위 nr_candidates 키워드를 선택하고 여기 중에 가장 유사성이 낮은 top_n개의 키워드를 선택!
max_sum_similarity(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['소방 드론 재난', '자격증 드론 전문가', '월간 드론 측정', '전망 드론 활용', '미세먼지 관리 드론']

In [15]:
"""
Maximal Marginal Relevance
MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력
문서와 가장 유사한 키워드/키프레이즈를 선택한 뒤, 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택
"""

def maximal_marginal_relevance(doc_embedding, candidate_embeddings, words, top_n, diversity):
    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
    
    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)
    
    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스 추출
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]
    
    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # --> candidates_idx = [0,1,3,4,5,6,7,8,9,10 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
    
    # 최고의 키워드는 이미 추출했으므로 top_n - 1번 만큼 반복.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)
        
        # MMR 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1,1)
        mmr_idx = candidates_idx[np.argmax(mmr)]
        
        # keywords & candidates 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)
        
    return [words[idx] for idx in keywords_idx]

In [16]:
maximal_marginal_relevance(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['미세먼지 관리 드론', '실시 드론 산업', '관리 드론 활용', '월간 드론 측정', '전망 드론 활용']

In [17]:
maximal_marginal_relevance(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.6)

['미세먼지 관리 드론', '사업 밀집', '재난 현장 영상', '수치 점검', '산악 수난']